In [4]:
import plotly.plotly as py
import plotly.graph_objs as go
from __future__ import division
import glob
import scipy.stats
import numpy as np
import matplotlib
import copy
import random 
import math
import os

dictHeadPose = {0:"X Euler Angle", 1: "Y Euler Angle", 2: "Z Euler Angle"}
dictAuTranslation = {0 : "AU01", 1 : "AU02", 2 :"AU04", 3:"AU05", 4:"AU06",5:"AU07",6:"AU09",7:"AU10",8:"AU12",9:"AU14",10:"AU15",
                     11:"AU17",12:"AU20",13:"AU23",14:"AU25",15:"AU26",16:"AU28",17:"AU45"}
                     
dictTraitTranslation = {0 : "Openness", 1 : "Conscientiousness", 2 : "Extraversion", 3 : "Agreeableness", 4 : "Neuroticism"}

photoData= {}

def getPhotoData():
    traitFile = open("C:/Users/rrenv/Desktop/Level4Project/averageRatings.csv", "r")
    for line in traitFile:
        line = line[:-1]
        values = line.split(",")
        traitValues = []
        if (values[0][:12] not in photoData):
            photoData[values[0][:12]] = []
            for trait in values[1:]:
                traitValues.append(float(trait))
            photoData[values[0][:12]].append(traitValues)
    traitFile.close()
    #print len(photoData)
    filesToGoThrough = glob.glob("C:/Users/rrenv/Desktop/Level4Project/OpenFace_0.2_win_x64/data/outputFiles/*.pts")
    for file in filesToGoThrough:
        auIntensities = []
        auActivations = []
        headPose = []
        filename = file.split("/")[-1][12:][:12]
        currentFile = open(file, "r")
        listOfLines = currentFile.readlines()
        for i in range(0,3):
            value=listOfLines[74].split(" ")[i]
            if (i == 2):
                headPose.append(float(value[0:-1]))
            else:
                headPose.append(float(value))
        photoData[filename].append(headPose)
        for i in range(0,17):
            value=listOfLines[82+i].split(" ")[1]
            auIntensities.append(float(value[0:-1]))
        photoData[filename].append(auIntensities)
        for i in range(0,18):
            value=listOfLines[102+i].split(" ")[1]
            auActivations.append(float(value))
        photoData[filename].append(auActivations)
        currentFile.close()


getPhotoData()  ########################## SETS UP THE DICTIONARY WHICH IS READ FROM FOR THE GRAPHS

def getMedian(trait):
    traitValues = []
    for photo in photoData:
        traitValues.append(photoData[photo][0][trait])
    print np.median(traitValues)

# X Euler angle is looking up or down,
# Y is looking to the left or right,
# Z is having your head tilted to either side

def getHeadPose(vector, trait):
    vectorValues = [] #Will be a list of either x, y, or z euler angles
    personalityRankings=[]
    photoFileDirectory = os.path.join(os.path.dirname('__file__'), 'PhotoData')
    filesToGoThrough = glob.glob(photoFileDirectory + '/*.pts')
    for file in filesToGoThrough:
        filename = file.split("/")[-1][10:]
        currentFile = open(file, "r")
        listOfLines = currentFile.readlines()
        value=listOfLines[74].split(" ")[vector]
        if (vector == 2):
            vectorValues.append(float(value[0:-1]))
        else:
            vectorValues.append(float(value))
        personalityRankings.append(getPersonalityRanks(trait, filename))
    currentFile.close()
    print vectorValues
    return vectorValues, personalityRankings
    
#Whether or not Action Units are Active

def getAuActivations(auvalue, trait):
    auBinaries = []
    roundedRankings = []
    photoFileDirectory = os.path.join(os.path.dirname('__file__'), 'PhotoData')
    filesToGoThrough = glob.glob(photoFileDirectory + '/*.pts')
    for file in filesToGoThrough:
        filename = file.split("/")[-1][10:]
        currentFile = open(file, "r")
        listOfLines = currentFile.readlines()
        value=int(listOfLines[102+auvalue].split(" ")[1])
        auBinaries.append(value)
        roundedRankings.append(getRoundedRanks(trait, filename))
        currentFile.close()
    return auBinaries, roundedRankings
  
#Continuous from 0 to 5 on a action unit's 'intensity' in its activation
                    
def getAuIntensities(auvalue, trait): #au
    auIntensities = []
    personalityRankings = []
    photoFileDirectory = os.path.join(os.path.dirname('__file__'), 'PhotoData')
    filesToGoThrough = glob.glob(photoFileDirectory + '/*.pts')
    for file in filesToGoThrough:
        filename = file.split("/")[-1][10:]
        currentFile = open(file, "r")
        listOfLines = currentFile.readlines()
        value=listOfLines[82+auvalue].split(" ")[1]
        auIntensities.append(float(value[0:-1]))
        personalityRankings.append(getPersonalityRanks(trait, filename))
        currentFile.close()
    return auIntensities, personalityRankings
    
#Getting all Personality Rankings

def getPersonalityRanks(traitNumber, fileName):
    traitFile = open("C:/Users/rrenv/Desktop/Level4Project/averageRatings.csv", "r")
    for line in traitFile:
        values = line.split(",")
        if (values[0] == fileName[0:-10]):
            return float(values[traitNumber+1])
 

def getRoundedRanks(traitNumber, fileName):
    traitFile = open("C:/Users/rrenv/Desktop/Level4Project/averageRatings.csv", "r")
    for line in traitFile:
        values = line.split(",")
        if (values[0] == fileName[0:-10]):
            value = float(values[traitNumber+1])
            return round(value)
    
def getExpectedValues(au, trait):
    total=0	
    auActive=0
    auNonActive=0
    numberOfMinusThree=0
    numberOfMinusTwo=0	
    numberOfMinusOne=0
    numberOfZero=0
    numberOfOne=0
    numberOfTwo=0
    numberOfThree=0
    numberOfFour=0
    for x in range(len(au)):
        if au[x] == 0:
            auNonActive = auNonActive + 1
            total += 1
        if au[x] == 1:
            auActive += 1
            total+=1
        if(trait[x]== -3):
            numberOfMinusThree += 1
        if(trait[x]==-2):
            numberOfMinusTwo += 1
        if(trait[x] == -1):
            numberOfMinusOne+=1
        if(trait[x] == 0):
            numberOfZero+=1
        if(trait[x] == 1):
            numberOfOne+=1
        if(trait[x] ==2):
            numberOfTwo+=1
        if(trait[x] ==3):
            numberOfThree+=1
        if(trait[x] ==3):
            numberOfFour+=1
    expectedZeroToMinusThree = numberOfMinusThree*(auNonActive/total)
    expectedOneToMinusThree= numberOfMinusThree*(auActive/total)
    expectedZeroToMinusTwo = numberOfMinusTwo*(auNonActive/total)
    expectedOneToMinusTwo = numberOfMinusTwo*(auActive/total)
    expectedZeroToMinusOne = numberOfMinusOne*(auNonActive/total)
    expectedOneToMinusOne = numberOfMinusOne*(auActive/total)
    expectedZeroToZero = numberOfZero*(auNonActive/total)
    expectedOneToZero = numberOfZero*(auActive/total)
    expectedZeroToOne = numberOfOne*(auNonActive/total)
    expectedOneToOne = numberOfOne*(auActive/total)
    expectedZeroToTwo = numberOfTwo*(auNonActive/total)
    expectedOneToTwo = numberOfTwo*(auActive/total)	
    expectedZeroToThree = numberOfThree*(auNonActive/total)
    expectedOneToThree = numberOfThree*(auActive/total)		
    expectedZeroToFour = numberOfFour*(auNonActive/total)
    expectedOneToFour = numberOfFour*(auActive/total)
    expectedValues = [expectedZeroToMinusThree, expectedOneToMinusThree, expectedZeroToMinusTwo, expectedOneToMinusTwo, expectedZeroToMinusOne,
								expectedOneToMinusOne, expectedZeroToZero, expectedOneToZero, expectedZeroToOne, expectedOneToOne, expectedZeroToTwo, expectedOneToTwo,
									expectedZeroToThree, expectedOneToThree, expectedZeroToFour, expectedOneToFour]
    return expectedValues
    
def getObservedValues(au, trait):
    numberOfZeroToMinusThree=0
    numberOfOneToMinusThree=0
    numberOfZeroToMinusTwo=0
    numberOfOneToMinusTwo=0
    numberOfZeroToMinusOne=0
    numberOfOneToMinusOne=0
    numberOfZeroToZero=0
    numberOfOneToZero=0
    numberOfZeroToOne=0
    numberOfOneToOne=0
    numberOfZeroToTwo=0
    numberOfOneToTwo=0
    numberOfZeroToThree=0
    numberOfOneToThree=0
    numberOfZeroToFour=0
    numberOfOneToFour=0
    for x in range(len(au)):
        if au[x] == 0:
            if trait[x] == -3:
                numberOfZeroToMinusThree += 1
            if trait[x] == -2:
                numberOfZeroToMinusTwo += 1
            if trait[x] == -1:
                numberOfZeroToMinusOne +=1 
            if trait[x] == 0:
                numberOfZeroToZero += 1
            if trait[x] == 1:
                numberOfZeroToOne += 1
            if trait[x] == 2:
                numberOfZeroToTwo += 1
            if trait[x] == 3:
                numberOfZeroToThree += 1
            if trait[x] == 4:
                numberOfZeroToFour += 1
        elif au[x] == 1:
            if trait[x] == -3:
                numberOfOneToMinusThree += 1
            if trait[x] == -2:
                numberOfOneToMinusTwo += 1
            if trait[x] == -1:
                numberOfOneToMinusOne += 1
            if trait[x] == 0:
                numberOfOneToZero += 1
            if trait[x] == 1:
                numberOfOneToOne += 1
            if trait[x] == 2:
                numberOfOneToTwo += 1
            if trait[x] == 3:
                numberOfOneToThree += 1
            if trait[x] == 4:
                numberOfOneToFour += 1
    return [numberOfZeroToMinusThree, numberOfOneToMinusThree, numberOfZeroToMinusTwo, numberOfOneToMinusTwo,
            numberOfZeroToMinusOne, numberOfOneToMinusOne, numberOfZeroToZero, numberOfOneToZero, numberOfZeroToOne,
            numberOfOneToOne, numberOfZeroToTwo, numberOfOneToTwo, numberOfZeroToThree, numberOfOneToThree,
            numberOfZeroToFour, numberOfOneToFour]
            

def getRoundedCounts(traitNumber):
    traitFile = open("C:/Users/rrenv/Desktop/Level4Project/averageRatings.csv", "r")
    listOfOccurrences = [0,0,0,0,0,0,0,0]
    for line in traitFile:
        values = line.split(",")
        value = round(float(values[traitNumber+1]))
        if (value == -3):
            listOfOccurrences[0] += 1
        elif (value == -2):
            listOfOccurrences[1] += 1
        elif (value == -1):
            listOfOccurrences[2] += 1
        elif (value == 0):
            listOfOccurrences[3] += 1
        elif (value == 1):
            listOfOccurrences[4] += 1
        elif (value == 2):
            listOfOccurrences[5] += 1
        elif (value == 3):
            listOfOccurrences[6] += 1
        else:
            listOfOccurrences[7] += 1
    return listOfOccurrences

for i in range (0,5):
    print "For the trait ", dictTraitTranslation[i]
    print getRoundedCounts(i), ' with Median: ', getMedian(i)

For the trait  Openness
[0, 25, 190, 271, 212, 125, 6, 0]  with Median:  0.272727272727
None
For the trait  Conscientiousness
[0, 16, 64, 131, 242, 299, 76, 1]  with Median:  1.36363636364
None
For the trait  Extraversion
[8, 137, 191, 182, 146, 110, 54, 1]  with Median:  -0.0909090909091
None
For the trait  Agreeableness
[9, 51, 146, 175, 197, 176, 75, 0]  with Median:  0.727272727273
None
For the trait  Neuroticism
[27, 187, 277, 196, 114, 27, 1, 0]  with Median:  -0.727272727273
None


In [24]:
#Making a Bubble chart for the correlation between AU Intensities and Personality Traits

listOfTraces = []

for i in range (0,17):
    myX = []
    myY = []
    myText = []
    mySizes = []
    myColours = []
    for j in range (0,5):
        auIntensities, traitRankings = getAuIntensities(16-i,j) #Assures that the listings are recorded from top down
        rho, pvalue = scipy.stats.spearmanr(auIntensities, traitRankings)
        myX.append(dictTraitTranslation[j])
        myY.append(dictAuTranslation[16-i])
        if (pvalue <= 0.00011764705):
            mySizes.append(abs(rho) * 1000) #Make the bubbles all scale up a bit
            if (rho > 0):
                myColours.append('rgb(0, 0, 255)')
            else:
                myColours.append('rgb(255, 0, 0)')
        elif (pvalue <= 0.00058823529):
            mySizes.append(abs(rho) * 1000)
            if (rho > 0):
                myColours.append('rgb(135,206,250)')
            else:
                myColours.append('rgb(255,192,203)')
        else:
            mySizes.append(None)
            myColours.append(None)
        
    trace = go.Scatter(
        x = myX,
        y = myY,
        #text = ,
        mode = 'markers',
        marker = dict(
            color = myColours,
            size = mySizes,  
            sizemode = 'area',
        )
    )
    listOfTraces.append(trace)
    
layout = go.Layout(
    height = 750,
    width = 500,
    showlegend=False,
    title='Action Unit Intensity and Personality Attribution',
)
data = listOfTraces
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='AU-Intensities-99Certainty')

In [22]:
##Now for AU activations

listOfTraces2 = []

for i in range (0,18):
    myX = []
    myY = []
    myText = []
    mySizes = []
    myColours = []
    for j in range (0,5):
        auBinaries, roundedRanks = getAuActivations(17-i,j) #Make top-down results 
        expectedValues = getExpectedValues(auBinaries, roundedRanks)
        observedValues = getObservedValues(auBinaries, roundedRanks)
        degOfFreedom = 6
        offset=0
        for x in range(0, len(expectedValues)-(1 + offset)):
            if (expectedValues[x-offset]==0.0):
                expectedValues.remove(expectedValues[x - offset])
                observedValues.remove(observedValues[x - offset])
                degOfFreedom -= 1
                offset+=1
        chisquare, pvalue = scipy.stats.chisquare(observedValues, expectedValues, degOfFreedom)
        myX.append(dictTraitTranslation[j])
        myY.append(dictAuTranslation[17-i])
        #myText.append()
        if (pvalue <= 0.0001111111):
            mySizes.append(abs(chisquare * 1.5)) #for now
            if (chisquare > 0):
                myColours.append('rgb(0, 0, 255)')
        elif (pvalue <= 0.00055555555):
            mySizes.append(abs(chisquare * 1.5)) #for now
            if (chisquare > 0):
                myColours.append('rgb(135,206,250)')
        else:
            mySizes.append('None')
            myColours.append('None')
        
    trace = go.Scatter(
        x = myX,
        y = myY,
        mode = 'markers',
        marker = dict(
            color = myColours,
            size = mySizes,  
            sizemode = 'area',
        )
    )
    listOfTraces2.append(trace)
    
layout = go.Layout(
    height = 750,
    width = 500,
    showlegend=False,
    title='Action Unit Activation and Personality Attribution',
)
data = listOfTraces2
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='AU-Activations-99Certainty')

In [6]:
##Now for Head Poses

listOfTraces3 = []

for i in range (0,3):
    myX = []
    myY = []
    myText = []
    mySizes = []
    myColours = []
    for j in range (0,5):
        vectorValues, personalityRankings = getHeadPose(i, j)
        rho, pvalue = scipy.stats.spearmanr(vectorValues, personalityRankings)
        myX.append(dictTraitTranslation[j])
        myY.append(dictHeadPose[i])
        if (pvalue <= (0.00066666666)):
            mySizes.append(abs(rho) * 1000)
            if (rho > 0):
                myColours.append('rgb(0, 0, 255)')
            else:
                #print 'helloooo'
                myColours.append('rgb(255, 0, 0)')
        elif (pvalue <= (0.00333333333)):
            mySizes.append(abs(rho) * 1000) 
            if (rho > 0):
                myColours.append('rgb(135,206,250)')
            else:
                #print 'helloooo'
                myColours.append('rgb(255,192,203)')
        else:
            mySizes.append(None)
            myColours.append(None)
        
    trace = go.Scatter(
        x = myX,
        y = myY,
        #text = myText,
        mode = 'markers',
        marker = dict(
            color = myColours,
            size = mySizes,  
            sizemode = 'area',
        )
    )
    listOfTraces3.append(trace)
    
layout = go.Layout(
    height = 250,
    width = 500,
    showlegend=False,
    title='Head Pose and Personality Attribution',
)
#data = listOfTraces3
#fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='HeadPose-99Certainty')

UnboundLocalError: local variable 'currentFile' referenced before assignment

In [23]:

#SET THE X-AXIS TO REPRESENT CATEGORIES
x = ['-3', '-2', '-1', '0', '1', '2', '3', '4']

trace1 = go.Bar(
    y=getRoundedCounts(0),
    x=x,
    name='Openness',
    marker=dict(
    )
)

trace2 = go.Bar(
    y=getRoundedCounts(1),
    x=x,
    name='Conscientiousness',
    marker=dict(
    )
)

trace3 = go.Bar(
    y=getRoundedCounts(2),
    x=x,
    name='Extraversion',
    marker=dict(
    )
)

trace4 = go.Bar(
    y=getRoundedCounts(3),
    x=x,
    name='Agreeableness',
    marker=dict(
    )
)

trace5 = go.Bar(
    y=getRoundedCounts(4),
    x=x,
    name='Neuroticism',
    marker=dict(
    )
)

data = [trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(
    title='Trait Distributions',
    xaxis=dict(
        title='Rounded Trait Ranking'
    ),
    yaxis=dict(
        title='Number Of Occurrences'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Trait-Distributions')

In [18]:
#EXTRA FUNCITON TO GET VALUES FOR HEAD POSE TABLE TO SHOW LACK OF DISTRIBUTION
def getHeadPoseMaxMin(vector):
    vectorValues = [] #Will be a list of either x, y, or z euler angles
    personalityRankings=[]
    maxValue = 0
    minValue = 0
    filesToGoThrough = glob.glob("C:/Users/rrenv/Desktop/Level4Project/OpenFace_0.2_win_x64/data/outputFiles/*.pts")
    for file in filesToGoThrough:
        filename = file.split("/")[-1][12:]
        currentFile = open(file, "r")
        listOfLines = currentFile.readlines()
        value=listOfLines[74].split(" ")[vector]
        if (vector == 2):
            vectorValues.append(float(value[0:-1]))
        else:
            vectorValues.append(float(value))
        if float(value) > maxValue:
            maxValue = float(value)
            maxFile = filename
        if float(value) < minValue:
            minValue = float(value)
            minFile = filename
        
    currentFile.close()
    print "max value: ", maxValue, "as seen in: ", maxFile, " min value: ", minValue, "as seen in: ", minFile
    return vectorValues

for i in range (0,3):
    getHeadPoseMaxMin(i)

max value:  0.276481 as seen in:  00408_940422_fa_det_0.pts  min value:  -0.357026 as seen in:  00480_940519_fa_det_0.pts
max value:  0.243978 as seen in:  00320_940422_fa_det_0.pts  min value:  -0.169511 as seen in:  00447_940422_fa_det_0.pts
max value:  0.145202 as seen in:  00451_940422_fa_det_0.pts  min value:  -0.188936 as seen in:  00316_940422_fa_det_0.pts
